In [1]:
#Imports
from load_hyperparams import load_hyperparams
from predict_satellites import get_satellite_properties
from predict_orphans import get_orphan_satellite_properties
from fix_lmc_coords import get_lmc_coords, rotate_about_LMC

In [2]:
#Set parameters
params = {}

params['alpha'] = -1.4
params['sigma_M'] = 0.1
params['M50'] = 8.
params['sigma_mpeak'] = 0.35
params['B'] = 1.
params['A'] = 34
params['sigma_r'] = 0.3
params['n'] = 1.3

In [3]:
#Load hyperparameters
hparams, cosmo_params, orphan_params, halo_data, vpeak_Mr_interp = load_hyperparams()

In [4]:
#Return satellite properties for a particular host halo
satellite_properties = get_satellite_properties(halo_data[0], params, hparams, cosmo_params, vpeak_Mr_interp)
print(satellite_properties)

{'Mr': array([-1.44076476e+01, -1.33070121e+01, -1.44816948e+01, -1.15540368e+01,
       -1.10651554e+01, -8.95218104e+00, -7.47215205e+00, -8.05524129e+00,
       -8.84255685e+00, -1.08249905e+01, -1.26001934e+01, -1.40072181e+01,
       -7.98673314e+00, -7.99917399e+00, -9.04516935e+00, -9.76702351e+00,
       -1.06788220e+01, -9.59247825e+00, -5.69891001e+00, -6.22652070e+00,
       -3.56338947e+00, -8.64994817e+00, -8.43897581e+00, -6.63195387e+00,
       -5.04877245e+00, -8.56473665e+00, -1.25081514e+01, -8.96124287e+00,
       -7.41602957e+00, -4.25653711e+00, -9.90831145e+00, -9.61021223e+00,
       -7.01205493e+00, -4.24317682e+00, -4.35885732e+00, -3.86625814e+00,
       -5.87369131e+00, -3.01170463e+00, -4.90829823e+00, -1.45259995e+00,
       -3.99483554e+00, -4.27944835e+00, -4.14701897e+00, -6.86443392e+00,
       -3.40787711e+00, -7.50098573e+00, -1.03747524e+00, -1.01720469e+00,
       -4.88622910e+00, -8.30074153e+00, -3.02253248e+00,  1.26846373e+00,
       -6.15123264

In [5]:
#Return orphan satellite properties for a particular host halo
orphan_satellite_properties = get_orphan_satellite_properties(halo_data[0], params, hparams, 
                                                              cosmo_params, vpeak_Mr_interp)
print(orphan_satellite_properties)

{'Mr': array([ -3.16391941,   2.22035947,  -0.45059312,   0.94148253,
         1.51092687,   1.79602606,   1.53360669,   0.62743586,
         2.98749152,   2.5397765 ,   3.02207031,   2.64538973,
         2.60348818,   2.81143583,   0.66972676,  -0.11821288,
         2.53991895,   2.48840854,  -0.98798875,   1.95627019,
         2.19748025,   0.15916661,   2.7577908 ,  -2.73833941,
        -2.96592973,   1.9714608 ,  -0.50152416,   2.76986623,
        -4.15645217,   2.54560672,  -1.43022859,   2.60567921,
        -3.91473845,   1.08600513,   2.48645024,   2.42542866,
         2.25637096,  -7.88799363,  -2.65644638,   2.35805483,
        -0.17004045,  -1.30383847,   2.92863918,  -1.4601017 ,
         2.70642529,   1.85709798,   1.02031429,   2.33687968,
         1.14629582,   2.01589779,  -1.84965063,   2.3770216 ,
         2.57371051,   2.9652934 ,  -1.22591206,   1.87356078,
         0.91830071,   2.83076748,  -0.79621707,   2.65824745,
         2.61344432,   1.54169912,  -6.28026983,

In [6]:
#Rotate about LMC
lmc_ind = 0
lmc_cartesian_coords = get_lmc_coords(halo_data[0],cosmo_params,lmc_ind)
satellite_properties_rotated = rotate_about_LMC(satellite_properties,halo_data[0],cosmo_params,
                                                lmc_cartesian_coords,lmc_ind,observer_ind=0)

In [13]:
#Precompute all observer locations
satellite_properties_list = []

for i in range(6):
    satellite_properties = get_satellite_properties(halo_data[0], params, hparams, cosmo_params, vpeak_Mr_interp)
    satellite_properties_list.append(rotate_about_LMC(satellite_properties,halo_data[0],cosmo_params,
                                                lmc_cartesian_coords,lmc_ind,i))